# Yelp Data Mining Project
## CMPE - 255 Data Mining Fall 2017
### Group 6
- Dhrumil Shah
- Nishant Rathi
- Rashmi Sharma

## Notebook to Preprocess Data and create CSR

In this Jupyter Notebook, we are loading Test and training data set we created from yelp data set.
This data is cleansed and converted to CSR Matrix. This CSR Matrix is huge hence storing it as pickle for further processing.

In [13]:
from util import *

## Converting full Data to CSR Matrix

In [14]:
with open("data/reviews_full.dat", "r") as fh:
    lines = fh.readlines()  
no_of_samples=50000
lines = [lines[i] for i in range(0,no_of_samples)]


In [15]:
lines[0]
print len(lines)

1000


In [16]:
userid = []
rating = []
docs = []
business = []
i = 0
j = 0

error_line_num = []
error_lines = []
for line in lines:
    try:
        i = i + 1
        l = line.split('\t', 4)
        userid.append(l[0])
        business.append(l[1])
        rating.append(l[2])
        #d = clean(l[3]).split()
        #kmers = getKmers(d)
        #d.extend(kmers)
        docs.append(l[3])
    except Exception as e:
        j = j + 1
        error_line_num.append(i)
        error_lines.append(line)
        print e

print 'Training Data: Number of lines processed: ' + str(i)
print 'Training Data: Length of userid array: ' + str(len(userid))
print 'Training Data: Length of rating array: ' + str(len(rating))
print 'Training Data: Length of docs array: ' + str(len(docs))
print 'Training Data: Length of business array: ' + str(len(business))
print 'Training Data: Number of exceptions encountered: ' + str(j)


Training Data: Number of lines processed: 1000
Training Data: Length of userid array: 1000
Training Data: Length of rating array: 1000
Training Data: Length of docs array: 1000
Training Data: Length of business array: 1000
Training Data: Number of exceptions encountered: 0


## Creating CSR Matrix for complete data

In [17]:
csr_mat = build_matrix(docs)
print 'Training CSR Formed'
mat1 = csr_idf(csr_mat, copy=True)
print 'Training IDF Formed'
docs_csr = csr_l2normalize(mat1, copy=True)
print 'Training L2Norm Formed'

Training CSR Formed
Training IDF Formed
Training L2Norm Formed


In [18]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
n_samples = 50000
n_features = 15000
n_components = 20
n_top_words = 20

data_samples = docs[:n_samples]

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))




# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)

tfidf_nmf = nmf.transform(tfidf)

print("done in %0.3fs." % (time() - t0))
print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")

vocab = np.array(tfidf_vectorizer.get_feature_names())


Extracting tf-idf features for NMF...
done in 0.226s.
Fitting the NMF model (generalized Kullback-Leibler divergence) with tf-idf features, n_samples=1000 and n_features=15000...
done in 3.799s.

Topics in NMF model (generalized Kullback-Leibler divergence):


In [19]:
save_pickle(tfidf_nmf, 'pickle/nmf.pickle')
save_pickle(vocab, 'pickle/vocab.pickle')

## Save Data to pickle

In [20]:
filename = 'pickle/'
save_pickle(userid, filename+'userid.pickle')
save_pickle(rating, filename+'rating.pickle')
save_pickle(business, filename+'business.pickle')
save_pickle(docs_csr, filename+'docs_csr.pickle')
print 'Training Data saved into pickle'

Training Data saved into pickle
